In [ ]:
from SynTemp.SynUtils.utils import load_from_pickle
data = load_from_pickle('./Data/test..pkl.gz')

In [ ]:
from SynTemp.SynRule.hierarchical_clustering import HierarchicalClustering
node_label_names = ["element", "charge"]
hcl = HierarchicalClustering(node_label_names=node_label_names,
                            node_label_default=["*", 0],
                            edge_attribute="order",
                            max_radius=3)

In [ ]:
reaction_dicts, templates = hcl.fit(data, 'ITSGraph', templates=None, update_template=True, root_sample=100)

In [ ]:
import pandas as pd
pd.DataFrame(reaction_dicts)

In [ ]:
a= []
a.append([1,2,3])
a

In [ ]:
len(templates[1])

In [ ]:
templates

In [ ]:
templates

In [ ]:
len(data)

In [ ]:
templates

In [ ]:
import pandas as pd
data = pd.DataFrame(reaction_dicts)

In [ ]:
len(templates[1])

In [ ]:
data['Cluster_R0'].value_counts()

In [ ]:
its_graphs = [value['ITSGraph'] for value in data]

In [ ]:
cluster_indices_0, temp_0 = hcl.process_level(its_graphs[:1000],0,nodeLabelNames=node_label_names,
                            nodeLabelDefault=["*", 0],
                            edgeAttribute="order", templates=None, update_template=True)
print(len(temp_0))

In [ ]:
cluster_indices_1, temp_1 = hcl.process_level(its_graphs[:1000],1,nodeLabelNames=node_label_names,
                            nodeLabelDefault=["*", 0],
                            edgeAttribute="order", templates=None, update_template=True)

print(len(temp_1))

In [ ]:
cluster_indices_2, temp_2 = hcl.process_level(its_graphs[:1000],2,nodeLabelNames=node_label_names,
                            nodeLabelDefault=["*", 0],
                            edgeAttribute="order", templates=None, update_template=True)

print(len(temp_1))

In [ ]:
cluster_indices_1, templates_1 = process_child_level(its_graphs, cluster_indices_0, node_label_names, radius=1)

In [ ]:
len(templates_1)

In [ ]:
len(templates_1)

In [ ]:
cluster_indices_2, templates_2 = process_graph_clusters(its_graphs, cluster_indices_1, node_label_names, radius=2)

In [ ]:
cluster_indices_3, templates_3 = process_graph_clusters(its_graphs, cluster_indices_2, node_label_names, radius=3)

In [ ]:
len(templates_1)

In [ ]:
graph_dict, index_dict = split_graphs_by_class_and_indices(its_graphs, cluster_indices_1)

In [ ]:
index_dict[76]

In [ ]:
bug = [value for key, value in enumerate(its_graphs) if key in index_dict[76]]

In [ ]:
len(bug)

In [ ]:
cluster_indices_2, templates_2 = process_graph_clusters(its_graphs, cluster_indices_1, node_label_names, radius=2)

In [ ]:
len(cluster_indices_1)

In [ ]:
len(its_graphs)

In [ ]:
len(cluster_indices_1)

In [ ]:

graph_dict, index_dict = split_graphs_by_class_and_indices(its_graphs[:1000], cluster_indices_0)
templates = []
cluster_indice_all = ['a' for i in range(len(its_graphs[:1000]))]
max_index_template = 0
for key, value in graph_dict.items():
    
    cluster_indices_batch, new_templates = hcl.process_level(value,2,nodeLabelNames=node_label_names,
                            nodeLabelDefault=["*", 0],
                            edgeAttribute="order", templates=None, update_template=True)
    cluster_indices_batch = [i + max_index_template for i in cluster_indices_batch]
    new_templates = [{'Cluster_id': value['Cluster_id']+max_index_template, 'RC': value['RC'], 'Parent': key} for value in new_templates]
    max_index_template += len(new_templates)
    templates.extend(new_templates)
    key_index = index_dict[key]

    for i, j in enumerate(key_index):
        cluster_indice_all[j] = cluster_indices_batch[i]
    

In [ ]:
cluster_indices_all, templates = process_graph_clusters(its_graphs, cluster_indices_0, node_label_names, radius=1)

In [ ]:
templates[-1]

In [ ]:
pd.DataFrame(cluster_indice_all)

In [ ]:
templates[-1]

In [ ]:
cluster_indices_batch

In [ ]:
len(templates)

In [ ]:
templates

In [ ]:
pd.concat([pd.DataFrame(cluster_indices_0), pd.DataFrame(cluster_indice_all)], axis=1)

In [ ]:
cluster_indice_all

In [ ]:
templates

In [ ]:
len(templates)

In [ ]:
cluster_indices_1[:10]

In [ ]:
cluster_indice_all[:10]

In [ ]:
pd.DataFrame(cluster_indices_1).value_counts()

In [ ]:
pd.DataFrame(cluster_indice_all).value_counts()

In [ ]:
#from SynTemp.SynRule.hierarchical_clustering import HierarchicalClustering
node_label_names = ["element", "charge"]
hcl = HierarchicalClustering(node_label_names=node_label_names,
                            node_label_default=["*", 0],
                            edge_attribute="order",
                            max_radius=1)

reaction_dicts, templates = hcl.fit(data[:])

In [ ]:
node_label_names = ["element", "charge"]
hcl = HierarchicalClustering(node_label_names=node_label_names,
                            node_label_default=["*", 0],
                            edge_attribute="order",
                            max_radius=1)

reaction_dicts, templates = hcl.fit(data[:], root_sample=1000)

In [ ]:
pd.DataFrame(reaction_dicts)

In [ ]:
from SynTemp.SynUtils.utils import load_from_pickle
data = load_from_pickle('./cluster.pkl.gz')

In [ ]:
data[0]

In [ ]:
data = [{'R-id':value['R-id'], 'Cluster_R0': value['Cluster_R0'],
         'Cluster_R1': value['Cluster_R1'], 'Cluster_R2': value['Cluster_R2'],
         'Cluster_R3': value['Cluster_R3']} for value in data]

id_key = [value['R-id'] for value in data]

In [ ]:
from SynTemp.SynUtils.utils import load_database
original = load_database('./Data/DPO/USPTO_50K/train.json.gz')
original_short = [value for value in original if value['R-id'] in id_key]

In [ ]:
import pandas as pd
all_data = pd.concat([pd.DataFrame(original_short), pd.DataFrame(data)], axis =1)